### BLAST: Basic Local Alignment Search Tool

Consiste numa ferramenta para comparar sequências biológicas, conjunto de algoritmos e ferramentas para procurar sequências similares em bases de dados de grande dimensão. 

**Como funciona o [BLAST?](https://edu.taugc.com/blog/voce-sabe-como-o-blast-funciona/)**

O algoritmo identifica similaridades entre sequências por meio de correspondências curtas. De seguida, ocorre a procura por alinhamentos locais utilizando conjuntos de um tamanho definifo pelo utilizador, normalmente é de 3 letras para aminoácidos e 11 para nucleótidos. 

* Exemplo:
    * GLFKA seria analisada em conjuntos de três letras: GLK, LKF, KFA.

Os alinhamentos locais são construídos a partir de correspondências comuns, servindo como ponto de partida para uma extensão em ambas as direções. Essa extensão continua até que o alinhamento atinja um valor de pontuação pré-definido. 

Nesta UC, foi-nos proposto a criação de um algoritmo BLAST versão simplificada que assenta em três pontos principais:
* Considera apenas matches perfeitos entre a query e as sequências da BD
* Critérios simples usados para a extensão dos hits
* Score será a contagem do nº de matches

* **Query map**

A primeira função criada, **query_map()**, procura mapear as sequências através de uma janela de tamanho **w** para identificar as subsequências e os seus índices correspondentes. 

Inicialmente, a função verifica se a sequência de entrada é do tipo "ADN" ou "sequência de aminoácidos", utilizando a função auxiliar **tipo_seq()**. Em caso de não ser nenhum desses tipos, a função levanta um erro. Além disso, verifica se o valor inserido em w é um inteiro positivo válido. Após realizar um aprimoramento na sequência para prepará-la para a análise subsequente, a função divide a sequência em todas as possíveis sub-sequências de tamanho w e armazena-as numa lista.

É então criado um dicionário vazio para armazenar as subsequências e seus índices correspondentes. A função percorre a sequência e, para cada sub-sequência de tamanho w, verifica se ela já está presente no dicionário. Se estiver, adiciona o índice atual à lista de índices associada a essa sub-sequência. Se não estiver, cria uma nova entrada no dicionário com a sub-sequência como chave e uma lista contendo o índice atual como valor.

Finalmente, a função retorna o dicionário resultante, que contém as subsequências como chaves e os índices onde essas subsequências são encontradas na sequência inserida.

In [20]:
def query_map(query, w): 
    """
    Mapeia as sequências numa janela  para identificar subsequências

    
    Parâmetros:
    -------------
    query : str 
        Sequência a ser mapeada
    w : int 
        Tamanho da janela

        
    Retorna:
    -------------
    dict: Dicionário de subsequências mapeadas com seus índices.


    Levanta:
    -------------
    ValueError
        Caso sequência inserida seja inválida ou tamanho da janela não ser um inteiro positivo

        
    """

    from scripts.auxiliares import tipo_seq
    from scripts.auxiliares import aprimorar_seq

    if not isinstance(query, str):
        raise TypeError("A sequência deve ser uma string")
    if not isinstance(w, int) or w <= 0:
        raise ValueError("O tamanho da janela deve ser um inteiro positivo")

    if tipo_seq(query) == "DNA":
        pass
    elif tipo_seq(query) == "Sequência de aminoácidos":
        pass
    else:
        raise ValueError("Sequência Inválida")
    
    query = aprimorar_seq(query)

    seq_dic = {}

    for i in range(len(query) - w + 1):
        subsequence = query[i:i + w]

        if subsequence in seq_dic:

            seq_dic[subsequence].append(i)
        else:
            seq_dic[subsequence] = [i]

    return seq_dic

**Exemplo**:

In [9]:
query_map("ATCGCTG", 3)

{'ATC': [0], 'TCG': [1], 'CGC': [2], 'GCT': [3], 'CTG': [4]}

**Testes de unidade**:

In [25]:
import unittest

class TestBLASTFunctions(unittest.TestCase):
    def test_query_map_vazio(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            query_map("", 3)

    def test_w_vazio(self):
        # Testa se a função lança um erro ao receber um valor vazio para w
        with self.assertRaises(ValueError):
            query_map("ACT", "")

    def test_query_numerico(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            query_map(1, 3)
            
suite = unittest.TestLoader().loadTestsFromTestCase(TestBLASTFunctions)
unittest.TextTestRunner(verbosity=3).run(suite)

test_query_map_vazio (__main__.TestBLASTFunctions) ... ok
test_query_numerico (__main__.TestBLASTFunctions) ... ok
test_w_vazio (__main__.TestBLASTFunctions) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.016s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

* **Hits**

De seguida, foi criada uma função denominada **hits()** que realiza a análise dos hits entre as subsequências identificadas na sequência de consulta (query) e a sequência da base de dados.

A função recebe dois parâmetros:

* dic_query: Um dicionário proveniente da função anterior, query_map(), que mapeia as subsequências e seus índices na sequência de consulta.
* seq: A sequência da base de dados a ser analisada.

A função percorre cada entrada no dicionário, onde cada chave representa uma subsequência e o valor é uma lista de índices onde essa subsequência ocorre na sequência de consulta. Para cada valor na lista de índices, a função verifica se a chave (subsequência) está presente na sequência da base de dados (seq). Se estiver, utiliza expressões regulares para encontrar todas as ocorrências da subsequência na base de dados, obtendo uma lista de índices dessas ocorrências.

Em seguida, a função cria tuplos contendo os índices da subsequência na sequência de consulta e os índices correspondentes na sequência da base de dados. Esses tuplos são adicionados a uma lista.

Finalmente, a função retorna uma lista de tuplos, onde cada tuplo representa um hit entre a sequência de consulta e a sequência da base de dados, contendo os índices onde a subsequência foi encontrada em ambas as sequências.

In [3]:
def hits(dic_query, seq):
    """ 
    Função que devolve lista de hits em que cada elemento é um tuplo com os índices na sequência query e na sequência da base da dados

    
    Parâmetros
    -------------
    dic_query : dict
        dicionário proveniente da função anterior, query_map()
    seq : str
        sequência da base de dados
    
        
    Returns
    -------------
    list
        lista de tuples com hits da sequência query e da sequência da base de dasos

    """
    # Adicionar tratamento de exceções para lidar com casos em que a chave não está presente na sequência
    import re

    lista_tuplos = []

    for chave, valor in dic_query.items():
        print(chave, valor)
        for valor_unico in valor:
            if chave in seq:
                lista_ocorrencias = [m.start() for m in re.finditer(chave, seq)]
                for index in lista_ocorrencias:
                    tuplo = (valor_unico, index)
                    lista_tuplos.append(tuplo)

    return lista_tuplos

In [17]:
def expande_direcao(alinhamento, sequencia, direcao):
    """
    Função para expandir o alinhamento numa direção específica.
    Recebe um alinhamento, a sequência original e a direção para expandir.
    Retorna a sequência expandida na direção especificada.

    Parâmetros
    -------------
    alinhamento : tuple
        Tuplo representando o alinhamento, por exemplo, (inicio, fim)
    sequencia : str
        Sequência original do alinhamento
    direcao : str
        Direção para expandir o alinhamento (por exemplo: 'esquerda', 'direita', 'ambas')

    Returns
    -------------
    str
        Sequência expandida na direção especificada
    """
    # Adicionar manipulação de exceções para garantir que os índices não ultrapassem os limites da sequência
    inicio, fim = alinhamento
    if direcao == 'esquerda':
        while inicio > 0 and sequencia[inicio-1] != '-':
            inicio -= 1
    elif direcao == 'direita':
        while fim < len(sequencia) - 1 and sequencia[fim+1] != '-':
            fim += 1
    elif direcao == 'ambas':
        while inicio > 0 and sequencia[inicio-1] != '-':
            inicio -= 1
        while fim < len(sequencia) - 1 and sequencia[fim+1] != '-':
            fim += 1

    return sequencia[inicio:fim+1]


In [4]:
def extended_hits(alinhamentos, sequencia, w, k, threshold):
    """
    Identifica hits estendidos com base em um limiar de similaridade.

    Args:
    alinhamentos (list): Lista de alinhamentos do BLAST.
    sequencia (str): Sequência original.
    w (int): Tamanho da janela.
    k (int): Fator de expansão.
    threshold (float): Limiar de similaridade.

    Returns:
    list: Lista de hits estendidos que atendem ao limiar de similaridade especificado.
    """
    # CRIAR a função calculate_similarity(subsequence, query) para calcular a similaridade, wunch ou waterman? se ja criada substituir pelo nome correto.
    expanded_regions = expanded_hits(alinhamentos, sequencia, w, k)
    similar_hits = []
    for start, end in expanded_regions:
        subsequence = sequencia[start:end]
        if calculate_similarity(subsequence, query) >= threshold:
            similar_hits.append((start, end))
    return similar_hits

In [5]:
def best_hits(hits, criterio='pontuacao', top_n=1):
    """
    Função alternativa para encontrar os melhores hits com base em critérios específicos.

    Parâmetros
    -------------
    hits : list
        Lista de tuplos representando os hits, por exemplo, [(indice_seq_query, indice_seq_db, pontuacao), ...]
    criterio : str, opcional
        Critério para selecionar os melhores hits (por exemplo: 'pontuacao', 'identidade', 'tamanho_alinhamento', etc.)
    top_n : int, opcional
        Número de melhores hits a serem retornados

    Returns
    -------------
    list
        Lista dos melhores hits do blast
    """
    if criterio == 'pontuacao':
        # Ordena os hits com base na pontuação
        hits_ordenados = sorted(hits, key=lambda x: x[2], reverse=True)
    elif criterio == 'identidade':
        # Ordena os hits com base na identidade
        # Substitua esta lógica com o cálculo da identidade
        hits_ordenados = sorted(hits, key=lambda x: calcular_identidade(x), reverse=True)
    # Verificar se a função calcular_identidade está definida em algum lugar do código, waterman ou wunch? Se ja criada substituir pelo nome correto.
    # Adiciona mais critérios aqui, se existirem?

    # Retornar os top N melhores hits
    return hits_ordenados[:top_n]

In [6]:
def expanded_hits(alinhamentos, sequencia, w, k):
    """
    Expande os hits do BLAST para identificar regiões similares na sequência original.

    Args:
    alinhamentos (list): Lista de alinhamentos do BLAST.
    sequencia (str): Sequência original.
    w (int): Tamanho da janela.
    k (int): Fator de expansão.

    Returns:
    list: Lista de regiões expandidas que correspondem aos hits do BLAST.
    """
    expanded_regions = []
    for hit in alinhamentos:
        start, end = hit  # assumindo que hit é um tuplo com posições de inicio e fim estabelecidas.
        expanded_start = max(0, start - k * w)
        expanded_end = min(len(sequencia), end + k * w)
        expanded_regions.append((expanded_start, expanded_end))

    return expanded_regions


In [7]:
def visualize_results(hits):  
    for hit in hits:
        print('Hit encontrado:', hit)

In [ ]:
import unittest

class TestBLASTFunctions(unittest.TestCase):
    def test_query_map_vazio(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(AssertionError):
            query_map("", 3)

    def test_w_vazio(self):
        # Testa se a função lança um erro ao receber um valor vazio para w
        with self.assertRaises(AssertionError):
            query_map("ACT", "")

    def test_query_numerico(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(AssertionError):
            query_map(1, 3)

    def test_hits(self):
        # Teste para a função hits
        dic_query = {'ATC': [0, 4], 'TCG': [1, 5], 'CGA': [2, 6]}
        seq = "ATCGATCGA"
        expected_result = [(0, 0), (0, 4), (1, 1), (1, 5), (2, 2), (2, 6)]
        self.assertEqual(hits(dic_query, seq), expected_result)

    def test_expanded_hits(self):
        # Teste para a função expanded_hits
        alinhamentos = [(3, 6), (8, 10)]
        sequencia = "ATCGATCGA"
        w = 3
        k = 2
        expected_result = [(0, 9), (5, 12)]
        self.assertEqual(expanded_hits(alinhamentos, sequencia, w, k), expected_result)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBLASTFunctions)
unittest.TextTestRunner(verbosity=3).run(suite)